# this is a notebook to process abstracts from the BCSWomen Lovelace Colloquium

In [ ]:
import pandas as pd
data=pd.read_excel('final.xls')
data=data.set_index('ID')
data.head()

# processing student data: 

Write out each abstract as a txt file

In [ ]:
from pathlib import Path
import shutil
import os

#if exists, remove directory
dirpath = Path('output')
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)
#create directory
os.mkdir('output')

n=0
#write files
for idex, abst in data.iterrows():
    contest=abst['Which poster contest are you entering?']
    title=abst['Poster Title']
    abstract=abst['Poster Abstract ( 250 words)']
    filename=f"output/abs{idex}.txt"
    with open(filename, "w") as f:
        f.write("abstract id: ")
        f.write(str(idex))
        f.write('\n')
        f.write(contest)
        f.write('\n\n')
        f.write(title)
        f.write('\n\n')
        f.write(abstract)
    n=n+1
print(f'Successfully written {n} abstract files')

# Processing reviewer data: next cell
this reads in the reviewer csv 
does some summary stats
creates zip files per reviewer

In [ ]:
assignment=pd.read_csv('reviewers.csv')
assignment=assignment.set_index('ID')
assignment.head()


In [ ]:
from collections import defaultdict
reviewers=defaultdict(list)
for idex, stud in assignment.iterrows():
    reviewers[stud['R1']].append(idex)
    reviewers[stud['R2']].append(idex)

In [ ]:
import zipfile as zf
for reviewer,li in reviewers.items():
    print(f"{reviewer} has {len(li)} abstracts")
    fn=f"{reviewer}.zip"
    with zf.ZipFile(fn, 'w') as myzip:
        for idex in li:
            filename=f"output/abs{idex}.txt"
            myzip.write(filename)

        

In [ ]:
x=list(reviewers.keys())
print(sorted(x))

In [ ]:
len(x)

In [ ]:
done=pd.read_csv('revs-19-feb.csv')
done=done.dropna()
done['Abstract ID']=done['Abstract ID'].astype(int)
done=done.rename(columns={'Feedback to student: please try to say something nice, something constructive, and a suggestion for presentation.  E.g. "This is a fascinating poster topic and very timely. You should take care to spell check your work before submission, as there are a few grammatical errors. Have you read [insert interesting paper name] on this? You might enjoy it. When designing your poster it would be nice to include images of [insert thing]"':'Feedback'})
done.head()

In [ ]:
has_reviews={}
reviewed_by={}
for idex, stud in assignment.iterrows():
    has_reviews[idex]=0
    reviewed_by[idex]=[]
    reviews_by_abs[idex]=[]
left={}
for reviewer,li in reviewers.items():
    print(f"{reviewer} has {len(li)} abstracts: {li}")
    print(f"{reviewer} has reviewed {len(done.loc[done['Reviewer name']==reviewer])} abstracts ")
    donelist=list(done['Abstract ID'].loc[done['Reviewer name']==reviewer])
    print(donelist)
    for abstr in donelist:
        has_reviews[abstr]+=1
        reviewed_by[abstr].append(reviewer)
    if (len(donelist)==len(li)):
        print("The reviewer has reviewed the right number of abstracts")
        if (donelist.sort()==li.sort()):
            print("The reviewed abstracts match the assigned abstracts")
        else:
            print("The reviewed abstracts don't match the assigned abstracts") 
    else:
        print(f"The reviewer ({reviewer})hasn't finished")
        unreviewed = [i for i in li if i not in donelist]
        left[reviewer]=unreviewed
print (left)

print(f"There are {sum(value == 0 for value in has_reviews.values())} abstracts with 0 reviews")

print(f"There are {sum(value == 1 for value in has_reviews.values())} abstracts with 1 review")

print(f"There are {sum(value == 2 for value in has_reviews.values())} abstracts finished")

# checking the leftover ones

In [ ]:
for i in left:
    for ab in left[i]:
        print(f"{ab} is from {data['University name e.g. Lancaster University'].iloc[ab]}")
        print(f"{ab} was reviewed by {reviewed_by[ab]}")

# finding those from a specific uni

In [ ]:
data[['Name and surname','University email']].loc[data['University name e.g. Lancaster University']=='Lancaster University']

# finding the problematic ones

In [ ]:
problems=done.loc[done['Overall']=='Problematic']
problems.drop(['Timestamp','Overall'], axis=1)
problems=problems.sort_values(by='Abstract ID')
for i,p in problems.iterrows():
    print(f"abstract {p['Abstract ID']} {p['Reviewer name']} says:")
    print(p['Feedback'])

# Make a list of the reject ones by id

In [ ]:
rejects=[4,8,14,97]

# generating the email text

In [ ]:
reviews_by_abs={}

for idex, stud in assignment.iterrows():
    reviews_by_abs[idex]=[]

for idx,rev in done.iterrows():
    reviews_by_abs[rev['Abstract ID']].append(rev['Feedback'])



In [ ]:
outdir='email_text'
dirpath = Path(outdir)
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)
#create directory
os.mkdir(outdir)

n=0
#write files
filenamesh=f"{outdir}/mail_all_students.sh"

with open(filenamesh,"w") as scriptfile: # file to hold the shell script which sends the emails
    for idex, abst in data.iterrows():
        if idex not in rejects: #ignoring rejected abstracts
            contest=abst['Which poster contest are you entering?']
            title=abst['Poster Title']
            abstract=abst['Poster Abstract ( 250 words)']
            name=abst['Name and surname']
            email=abst['University email']
            filename=f"{outdir}/mail{idex}.txt"

            
            scriptfile.write(f"mail -s \"BCSWomen Lovelace Colloquium Acceptance\" {email} < {filename}\n")

            with open(filename, "w") as f:
                f.write(f"Dear {name}\n ")
                f.write('\n')
                f.write(f"Thank you for submitting an abstract to the {contest} \n")
                f.write("contest at the BCSWomen Lovelace Colloquium")
                f.write("\n\n")
                f.write("We are pleased to accept your abstract and invite you to present a poster\n")
                f.write("at the colloquium.")
                f.write("\n\n")
                f.write("Here are some comments from our reviewers:\n")
                for rno,fb in enumerate(reviews_by_abs[idex]):
                    f.write(f"Review text {rno}\n\n")
                    f.write(fb)
                    f.write('\n\n')
                f.write("\nWhat to do next:\n\n")
                f.write(" 1. Make sure you have the dates in your diary: \n")
                f.write("    30th March 1 day conference, with evening social on the 29th\n\n")
                f.write(" 2. Respond to this email confirming attendance\n\n")
                f.write(" 3. Start working on your poster\n\n")
                f.write(" 4. Join one (or both) of our groups for attendees at the event\n")
                f.write("    Facebook group: https://www.facebook.com/groups/3642085902553682\n")
                f.write("    Discord server: https://discord.gg/hTTuWD3E\n\n")
                f.write(" 5. This year as we are online, we'd like a short video or audio presentation\n")
                f.write("    The purpose of the audio/video is to allow us all to be more relaxed on the\n")
                f.write("    day, as we'll do the poster contest stuff beforehand.\n\n")
                f.write("To help with preparation we have an info sheet online here:\n\n")
                f.write("https://docs.google.com/document/d/1WkhxjhZaJ3aq9RJbeyT1xj9njGTTuYMdcwpeeG5J9kU/edit?usp=sharing\n")
                f.write("This is a live document and we'll keep it up to date as the event approaches.\n\n")
                f.write('As a reminder to you, your title and abstract are appended to this email. ')
                f.write('If your reviews suggest that the abstract would benefit from revising, then ')
                f.write('feel free to email me a new version')
                f.write('\n\nYours,\n\nHannah Dee\nBCSWomen Lovelace Colloquium Chair')

                f.write('\n\n')
                f.write(title)
                f.write('\n\n')
                f.write(abstract)
            n=n+1
print(f'Successfully written {n} email files')
print("REMEMBER TO EMAIL THE REJECTED STUDENTS BY HAND")
